<a href="https://colab.research.google.com/github/laurelkeys/machine-learning/blob/master/assignment-4/Trajectories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# PATH_TO_DATA = os.path.join("drive", "My Drive", "unicamp", "MC886", "atari")

import os
PATH_TO_DATA = ""

In [0]:
SAVE_DIR = os.path.join(PATH_TO_DATA, "data")
os.makedirs(SAVE_DIR, exist_ok=True)

SAVE_DIR # where the trajectories for each game will be saved to

'data'

In [0]:
LOG_DIR = os.path.join(PATH_TO_DATA, "data", "results")
os.makedirs(LOG_DIR, exist_ok=True)

LOG_DIR # where the stats for each game will be saved to

'data/results'

## Generate a dataset of trajectories from pre-trained RL agents on [Atari](https://gym.openai.com/envs/#atari) [environments](https://github.com/openai/gym/wiki/Table-of-environments).
That is, by the end of this notebook we will have $observation \rightarrow action$ mappings, where $observation$s are images of shape `IMG_SHAPE` and $action$s are integer values in the range $[0, 18)$, meaning:

| 0 | 1 | 2 | 3 | 4 | 5 |
| --- | --- | --- | --- | --- | --- |
| NOOP | FIRE | UP | RIGHT | LEFT | DOWN |


| 6 | 7 | 8 | 9 |
| --- | --- | --- | --- |
| UPRIGHT | UPLEFT | DOWNRIGHT | DOWNLEFT |


| 10 | 11 | 12 | 13 |
| --- | --- | --- | --- |
| UPFIRE | RIGHTFIRE | LEFTFIRE | DOWNFIRE |


| 14 | 15 | 16 | 17 |
| --- | --- | --- | --- |
| UPRIGHTFIRE | UPLEFTFIRE | DOWNRIGHTFIRE | DOWNLEFTFIRE |

In [0]:
# number of trajectories to generate
N_OF_TRAJECTORIES = 2

# number of steps per trajectory
N_OF_STEPS = 20

# list of string tuples in the format (RL Algorithm, Game Environment)
GAMES = [
    ("PPO2", "BreakoutNoFrameskip-v4"),
    ("PPO2", "PongNoFrameskip-v4"),
]

In [0]:
[env_id for algo, env_id in GAMES]

['BreakoutNoFrameskip-v4', 'PongNoFrameskip-v4']

## Install dependencies

Note that we're not installing [MPI](https://mpi4py.readthedocs.io/en/stable/), so the following algorithms will probably not work: `DDPG`, `GAIL`, `PPO1`, `TRPO`.

In [0]:
!apt-get update                                                  > /dev/null 2>&1
!apt-get install swig cmake zlib1g-dev ffmpeg freeglut3-dev xvfb > /dev/null 2>&1
!pip install pytablewriter                                       > /dev/null 2>&1
# !pip install pytablewriter pyyaml optuna scikit-optimize         > /dev/null 2>&1

In [0]:
#### Stable Baselines only supports TF 1.x for now ####
try:
    # Colab only
    %tensorflow_version 1.x
except Exception:
    pass

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.15.0


In [0]:
import os
from time import time
from IPython.display import clear_output

import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# NOTE use tqdm.write() instead of print() inside of tqdm wrapped loops
from tqdm import tqdm

import gym
from gym.envs.atari.atari_env import ACTION_MEANING

### Update [Stable Baselines](https://github.com/hill-a/stable-baselines) and clone [RL Zoo Baselines](https://github.com/araffin/rl-baselines-zoo)

In [0]:
!pip list | grep baselines

stable-baselines         2.2.1      


In [0]:
!yes | pip uninstall stable-baselines                           > /dev/null 2>&1
!pip install git+https://github.com/hill-a/stable-baselines.git > /dev/null 2>&1

In [0]:
!pip list | grep baselines

stable-baselines         2.9.0a0    


In [0]:
from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.common.vec_env import VecFrameStack, DummyVecEnv

# HACK to save logs
from stable_baselines import logger
os.environ["OPENAI_LOG_FORMAT"] = 'csv' # 'stdout,log,csv,tensorboard'
os.environ["OPENAI_LOGDIR"] = os.path.abspath(LOG_DIR)
logger.configure()

# NOTE add more algorithms here if you want to use them
from stable_baselines import PPO2, ACER, ACKTR
ALGO_IMPL = {
    'PPO2': PPO2,
    'ACER': ACER,
    'ACKTR': ACKTR,
}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
!git clone https://github.com/araffin/rl-baselines-zoo.git      > /dev/null 2>&1

## Load the pre-trained agents

In [0]:
!ls rl-baselines-zoo/trained_agents/

a2c  acer  acktr  ddpg	dqn  her  ppo2	sac  td3  trpo


In [0]:
PATH_TO_AGENTS = os.path.join("rl-baselines-zoo", "trained_agents")

In [0]:
ext = "NoFrameskip-v4.pkl"
# check the available pre-trained models
algorithms = ["PPO2"]
for algo in algorithms:
    algo_path = os.path.join(PATH_TO_AGENTS, algo.lower())
    print(algo_path + '/')
    for f in sorted(os.listdir(algo_path), key=lambda x: x[::-1]):
        # sort by the reverse filename, so env types get grouped together
        if f.endswith(ext):
            print("├──", f)

rl-baselines-zoo/trained_agents/ppo2/
├── PongNoFrameskip-v4.pkl
├── MsPacmanNoFrameskip-v4.pkl
├── EnduroNoFrameskip-v4.pkl
├── BeamRiderNoFrameskip-v4.pkl
├── SpaceInvadersNoFrameskip-v4.pkl
├── QbertNoFrameskip-v4.pkl
├── SeaquestNoFrameskip-v4.pkl
├── BreakoutNoFrameskip-v4.pkl


In [0]:
for i in range(2):
    clear_output() # HACK to remove TensorFlow warnings
    for algo, env_id in GAMES:
        print(f"('{algo}', '{env_id}')")
        agent_path = os.path.join(PATH_TO_AGENTS, algo.lower(), env_id + '.pkl')
        model = ALGO_IMPL[algo].load(agent_path, verbose=0)
        print("observation_space:", model.observation_space)
        print("action_space:", model.action_space)
        print()

('PPO2', 'BreakoutNoFrameskip-v4')
observation_space: Box(84, 84, 4)
action_space: Discrete(4)

('PPO2', 'PongNoFrameskip-v4')
observation_space: Box(84, 84, 4)
action_space: Discrete(6)



In [0]:
VERBOSE = 2 # 0, 1 or 2

In [0]:
print("N_OF_STEPS:", N_OF_STEPS)
print("N_OF_TRAJECTORIES:", N_OF_TRAJECTORIES)
print(N_OF_STEPS, "*", N_OF_TRAJECTORIES, "=", N_OF_STEPS * N_OF_TRAJECTORIES)

N_OF_STEPS: 20
N_OF_TRAJECTORIES: 2
20 * 2 = 40


## Use Stable Baseline's `generate_expert_traj`

In [0]:
TRAJ_IMAGES_FOLDER = "images" # name of the folder in which to save the observations
TRAJ_FILE_NAME = "trajectory" # name of the .npz trajectory file

In [0]:
print(LOG_DIR)
print(logger.get_dir())

data/results
/content/data/results


In [0]:
from stable_baselines.gail import generate_expert_traj

time_start = time()
print("================")
for algo, env_id in GAMES:
    time_start_env = time()

    env = make_atari_env(env_id, num_env=1, seed=0)
    env = VecFrameStack(env, n_stack=4) # Frame-stacking with 4 frames
    agent_path = os.path.join(PATH_TO_AGENTS, algo.lower(), env_id + '.pkl')
    
    print(f"('{algo}', '{env_id}')")
    print(f"Getting pre-trained agent from: '{agent_path}'")
    if VERBOSE > 1:
        print(f"env.envs: {env.envs}")
    print()
    
    model = ALGO_IMPL[algo].load(agent_path, env)
    
    traj_name = f"{env_id}_{algo}_{N_OF_STEPS}s"
    generate_expert_traj(model, env=env, 
                         n_episodes=N_OF_TRAJECTORIES, 
                         save_path=os.path.join(SAVE_DIR, traj_name, TRAJ_FILE_NAME), 
                         image_folder=TRAJ_IMAGES_FOLDER)

    env.close()
    print(f"\nΔt = {(time() - time_start_env):.2f}s")
    print("================")

print(f"\nTotal Δt = {(time() - time_start):.2f}s")

('PPO2', 'BreakoutNoFrameskip-v4')
Getting pre-trained agent from: 'rl-baselines-zoo/trained_agents/ppo2/BreakoutNoFrameskip-v4.pkl'
env.envs: [<ClipRewardEnv<WarpFrame<FireResetEnv<EpisodicLifeEnv<Monitor<MaxAndSkipEnv<NoopResetEnv<TimeLimit<AtariEnv<BreakoutNoFrameskip-v4>>>>>>>>>>]

Images will be recorded to data/BreakoutNoFrameskip-v4_PPO2_20s/images/
Image shape: (84, 84, 4)
actions (2182, 1)
obs (2182,)
rewards (2182, 1)
episode_returns (2,)
episode_starts (2182,)

Δt = 8.31s
('PPO2', 'PongNoFrameskip-v4')
Getting pre-trained agent from: 'rl-baselines-zoo/trained_agents/ppo2/PongNoFrameskip-v4.pkl'
env.envs: [<ClipRewardEnv<WarpFrame<FireResetEnv<EpisodicLifeEnv<Monitor<MaxAndSkipEnv<NoopResetEnv<TimeLimit<AtariEnv<PongNoFrameskip-v4>>>>>>>>>>]

Images will be recorded to data/PongNoFrameskip-v4_PPO2_20s/images/
Image shape: (84, 84, 4)
actions (3400, 1)
obs (3400,)
rewards (3400, 1)
episode_returns (2,)
episode_starts (3400,)

Δt = 11.97s

Total Δt = 20.28s


## Download generated dataset

In [0]:
trajectories = []
image_folders = []
for save_folder in os.listdir(SAVE_DIR):
    for x in os.listdir(os.path.join(SAVE_DIR, save_folder)):
        path = os.path.join(SAVE_DIR, save_folder, x)
        if x == TRAJ_IMAGES_FOLDER:
            image_folders.append(path)
        elif os.path.splitext(x)[0] == TRAJ_FILE_NAME:
            trajectories.append(path)

assert len(image_folders) == len(trajectories), f"{len(image_folders)} != {len(trajectories)} (len(image_folders) != len(trajectories))"

print('\n\n'.join([traj + '\n' + img_folder for traj, img_folder in zip(trajectories, image_folders)]))

data/BreakoutNoFrameskip-v4_PPO2_20s/trajectory.npz
data/BreakoutNoFrameskip-v4_PPO2_20s/images

data/PongNoFrameskip-v4_PPO2_20s/trajectory.npz
data/PongNoFrameskip-v4_PPO2_20s/images


In [0]:
x = np.load('data/PongNoFrameskip-v4_PPO2_20s/trajectory.npz', 
            allow_pickle=True, mmap_mode='r')
print(x.files)

['actions', 'obs', 'rewards', 'episode_returns', 'episode_starts']


In [0]:
def print_trajectory_info(x):
    # actions taken
    print("- actions:", x['actions'].shape)
    print("  actions taken:", ', '.join([ACTION_MEANING[action] for action in set(x['actions'].reshape(-1))]))
    # path to the observed images
    print("- obs:", x['obs'].shape)
    # reward for each step
    print("- rewards:", x['rewards'].shape)
    print("  reward values:", ', '.join([str(r) for r in set(x['rewards'].reshape(-1))]))
    # reward for each trajectory
    print("- episode_returns:", x['episode_returns'].shape)
    print("  episode returns", x['episode_returns'])
    # `done` value returned by env.step(action)
    print("- episode_starts:", x['episode_starts'].shape)
    print("  episode starts:", [i for i, ep_start in enumerate(x['episode_starts']) if ep_start])

In [0]:
print("================")
for trajectory in trajectories:
    print(trajectory)
    print_trajectory_info(np.load(trajectory, allow_pickle=True, mmap_mode='r'))
    print("================")

data/BreakoutNoFrameskip-v4_PPO2_20s/trajectory.npz
- actions: (2182, 1)
  actions taken: NOOP, FIRE, UP, RIGHT
- obs: (2182,)
- rewards: (2182, 1)
  reward values: 0.0, 1.0
- episode_returns: (2,)
  episode returns [96.  1.]
- episode_starts: (2182,)
  episode starts: [0, 2033]
data/PongNoFrameskip-v4_PPO2_20s/trajectory.npz
- actions: (3400, 1)
  actions taken: NOOP, FIRE, UP, RIGHT, LEFT, DOWN
- obs: (3400,)
- rewards: (3400, 1)
  reward values: 0.0, 1.0
- episode_returns: (2,)
  episode returns [21. 21.]
- episode_starts: (3400,)
  episode starts: [0, 1691]


## Evaluate agents
Let's evaluate a few trajectories of each game from `GAMES` to get a sense of how the agents are performing

In [0]:
# number of trajectories to evaluate each agent on
N_OF_EVAL_TRAJECTORIES = 2

results = {
    'algo': [],
    'env_id': [],
    'mean_reward': [],
    'std_reward': [],
    'n_timesteps': [],
    'n_episodes': []
}

TENSORBOARD_LOG = False
# https://stable-baselines.readthedocs.io/en/master/guide/tensorboard.html

if TENSORBOARD_LOG:
    %load_ext tensorboard
    %tensorboard --logdir tmp/tb_logs

In [0]:
time_start = time()
print("================")
for algo, env_id in GAMES:
    time_start_env = time()

    env = make_atari_env(env_id, num_env=1, seed=0)
    env = VecFrameStack(env, n_stack=4) # Frame-stacking with 4 frames
    agent_path = os.path.join(PATH_TO_AGENTS, algo.lower(), env_id + '.pkl')
    
    print(f"('{algo}', '{env_id}')")
    print(f"Getting pre-trained agent from: '{agent_path}'")
    if VERBOSE > 1:
        print(f"env.envs: {env.envs}")
    print()
    
    model = ALGO_IMPL[algo].load(agent_path, env)
    if TENSORBOARD_LOG:
        model.tensorboard_log = os.path.join(LOG_DIR, f"tb_logs")
        print(f"Adding TensorBoard logs to '{model.tensorboard_log}/'")
    
    for trajectory in tqdm(range(N_OF_EVAL_TRAJECTORIES), position=0, leave=True):
        # episode stats
        ep_len, ep_reward, ep_rewards = 0, 0.0, []

        obs = env.reset() # (84, 84, 4)
        for step in range(N_OF_STEPS):
            action = model.predict(obs)
            # clip action to avoid out of bound errors
            if isinstance(env.action_space, gym.spaces.Box):
                action = np.clip(action, env.action_space.low, env.action_space.high)
            
            obs, reward, done, infos = env.step(action)
            # NOTE action, reward and done are arrays since we're using a vectorized env
            _env = env.envs[0]
            # tqdm.write(f"env.envs[0].episode_rewards: {env.envs[0].episode_rewards}")
            # tqdm.write(f"env.envs[0].episode_lengths: {env.envs[0].episode_lengths}")
            # tqdm.write(f"env.envs[0].episode_times: {env.envs[0].episode_times}")
            # tqdm.write(f"env.envs[0].rewards: {env.envs[0].rewards}")

            # NOTE the return reward is not the Atari score
            #      so we have to get it from the infos dict
            ep_infos = infos[0].get('episode')
            if ep_infos is not None:
                tqdm.write(f"\nAtari Episode Score: {ep_infos['r']:.2f}")
                tqdm.write(f"Atari Episode Length: {ep_infos['l']}")
                #tqdm.write(f"\nAtari Episode Score: {round(sum(_env.rewards), 6):.2f}")
                #tqdm.write(f"Atari Episode Length: {len(_env.rewards)}")
            
            # FIXME ep_infos is always None 
            # try checking stable-baselines' Monitor.step()
            if infos is not None: # debug
                # if step == 0:
                #     tqdm.write(f"(DEBUG) step == 0: {infos}")
                if len(infos) > 1:
                    tqdm.write(f"(DEBUG) infos: {infos}")
                elif len(infos[0].keys()) > 1:
                    if 'terminal_observation' not in infos[0].keys() or len(infos[0].keys()) > 2:
                        tqdm.write(f"(DEBUG) infos[0]: {infos[0]}")

            ep_len += 1
            ep_reward += reward[0]
            if done:
                obs = env.reset()
                ep_rewards.append(ep_reward)
                if VERBOSE > 1:
                    tqdm.write(f"\nEpisode Reward: {ep_reward:.2f}")
                    tqdm.write(f"Episode Length: {ep_len}")
                ep_reward = 0.0
                ep_len = 0
        
        if VERBOSE > 0:
            tqdm.write("\nMean reward: {:.2f}, len(ep_rewards) == {}".format(
                       np.mean(ep_rewards) if len(ep_rewards) > 0 else 0.0, 
                       len(ep_rewards)))

    env.close()
    print(f"Δt = {(time() - time_start_env):.2f}s")
    print("================")

print(f"Total Δt = {(time() - time_start):.2f}s")

In [0]:
!ls data/results/

## Generate trajectories

Note that we use `make_atari_env` + `VecFrameStack` for `NoFrameskip-v4` environments, so each frame is converted to grayscale and downscaled from 210x160 to 84x84. Therefore, the $observation$ shape is `(84, 84, 4)` (four stacked frames), and **not** `(210, 160, 3)`, nor `(84, 84, 1)`.

In [0]:
PRINT_EARLY_DONE = False
PRINT_ACTIONS_TAKEN = False

PRINT_EVERY_N_TRAJECTORIES = N_OF_TRAJECTORIES // 10
# uncomment below not to print
# PRINT_EVERY_N_TRAJECTORIES = N_OF_TRAJECTORIES + 1

**TODO: evaluate the trajectories before saving the final datasets**  
**TODO: add button to load from data/ or save to drive**

In [0]:
time_start = time()
print("PRINT_EVERY_N_TRAJECTORIES:", PRINT_EVERY_N_TRAJECTORIES)
print("N_OF_TRAJECTORIES:", N_OF_TRAJECTORIES)
print("N_OF_STEPS:", N_OF_STEPS)
print("================")
for algo, env_id in GAMES:
    time_start_env = time()

    env = make_atari_env(env_id, num_env=1, seed=0)
    env = VecFrameStack(env, n_stack=4) # Frame-stacking with 4 frames
    agent_path = os.path.join(PATH_TO_AGENTS, algo.lower(), env_id + '.pkl')
    
    print(f"('{algo}', '{env_id}')")
    print(f"Getting pre-trained agent from: '{agent_path}'\n")
    
    model = ALGO_IMPL[algo].load(agent_path, env)
    
    for trajectory in tqdm(range(N_OF_TRAJECTORIES), position=0, leave=True):
        # store the "obs -> action" mapping
        observed_states, actions_taken = [], []

        obs = env.reset() # (84, 84, 4)
        for step in range(N_OF_STEPS):
            action = model.predict(obs)
            observed_states.append(obs)
            actions_taken.append(action)
            obs, reward, done, infos = env.step(action)
            if done:
                obs = env.reset()
                if PRINT_EARLY_DONE:
                    print(f"done at step {step + 1} (reseting env)")
        
        # NOTE action, reward and done are arrays since we're using a vectorized env
        observed_states = [obs[0] for obs in observed_states]
        actions_taken = [action[0][0] for action in actions_taken]
        
        np.savez_compressed(file=os.path.join(SAVE_DIR, f"{env_id}_{algo}_t{trajectory+1}_{N_OF_STEPS}s"), 
                            observations=observed_states, actions=actions_taken)
        
        if (trajectory + 1) % 10 == 0:
            print(f" Saved trajectory {trajectory+1} (of {N_OF_TRAJECTORIES})")

        if PRINT_ACTIONS_TAKEN and trajectory == N_OF_TRAJECTORIES - 1:
            print("\nActions taken:", ", ".join([ACTION_MEANING[action] for action in set(actions_taken)]))

    del observed_states
    del actions_taken
    env.close()
    print(f"Δt = {(time() - time_start_env):.2f}s")
    print("================")

print(f"Total Δt = {(time() - time_start):.2f}s")

In [0]:
trajectory_filenames = []
for r, ds, fs in os.walk(SAVE_DIR): # r=root, d=directories, f=files
    print(r + '/')
    for f in fs:
        print("|___", f)
        trajectory_filenames.append(f)

In [0]:
test_trajectory_filename = trajectory_filenames[0]
print(f"Loading from '{test_trajectory_filename}'\n")

test_trajectory_load = np.load(os.path.join(SAVE_DIR, test_trajectory_filename), 
                               allow_pickle=True)

print("observations shape:", test_trajectory_load['observations'].shape)
print("actions shape:", test_trajectory_load['actions'].shape)

In [0]:
# https://github.com/araffin/rl-baselines-zoo/blob/master/utils/record_video.py
# https://github.com/araffin/rl-baselines-zoo/blob/master/enjoy.py
# https://github.com/hill-a/stable-baselines#try-it-online-with-colab-notebooks-

## Old

In [0]:
# def save_as_image(observation, save_dir, img_name, prefix="img_", downscale=False):
#     # downscaling the image
#     if downscale:
#         im_array = cv2.resize(observation, INP_IMAGE_SHAPE) # TODO test tf.image.resize
#         im_array = np.array(im_array, dtype='float32')
#         im_array = (im_array/127.5) - 1
#         im = PIL.Image.fromarray(im_array, 'RGB')
#     else:
#         try:
#             im = PIL.Image.fromarray(observation, 'RGB')
#         except:
#             print(type(observation))
#     imname = "{}{}.png".format(prefix, img_name)
#     im.save(os.path.join(save_dir, imname))

In [0]:
# # you can change the default values here
# save_dir = SAVE_DIR
# num_images = IMAGES_TO_GENERATE

In [0]:
# os.makedirs(save_dir, exist_ok=True)

In [0]:
# envs = [gym.make(env_id) for env_id in ENV_IDS]

In [0]:
# for env_id, env in zip(ENV_IDS, envs):
#     print(env_id)
#     env_dir = os.path.join(save_dir, f"{env_id}_{IMAGES_TO_GENERATE}")
#     os.makedirs(env_dir, exist_ok=True)
    
#     env.reset()
#     i, current_env_images = 0, 0
    
#     actions_taken = []
#     while i < num_images:
#         # take a random action (sampled from the action space)
#         action = env.action_space.sample()
#         actions_taken.append(action)
#         assert 0 <= action < 18, f"action = {action}"
#         obs, _, done, _ = env.step(action)
#         if np.mean(obs) > 0.01:
#             save_as_image(obs, env_dir, str(i))
#             i += 1
#         else:
#             print("should I have been reached?")
#             continue
#         if done:
#             print(f"reseting {env_id} at i={i}")
#             env.reset()
    
#     actions_taken = np.asarray(actions_taken, dtype='int8')
#     print(actions_taken.shape, actions_taken.size, actions_taken.dtype)
#     np.save(os.path.join(save_dir, f"{env_id}_{IMAGES_TO_GENERATE}_actions"), actions_taken)

In [0]:
# IMG_SIZE = 160 # All images will be resized to 160x160

# def load_image(image_path):
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_png(image, channels=3)
#     image = tf.cast(image, tf.float32)
#     image = (image/127.5) - 1
#     image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
#     return image, image_path

# IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# # Create the base model from the pre-trained model MobileNet V2
# base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# s = time()
# # Get unique images
# encode_train = img_name_vector

# # Feel free to change batch_size according to your system configuration
# image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
# image_dataset = image_dataset.map(
#   load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# print((time()-s)/1000)

# for img, path in image_dataset:
#   batch_features = image_features_extract_model(img)
#   batch_features = tf.reshape(batch_features,
#                               (batch_features.shape[0], -1, batch_features.shape[3]))

#   for bf, p in zip(batch_features, path):
#     path_of_feature = p.numpy().decode("utf-8")
#     np.save(path_of_feature, bf.numpy())